In [342]:
import requests
import pandas as pd
import io
import numpy as np
import os
import warnings

#!pip install fuzzywuzzy
import fuzzywuzzy

from fuzzywuzzy import fuzz
import re

warnings.filterwarnings("ignore")
import spacy
nlp = spacy.load("en_core_web_sm")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [343]:
directory =os.makedirs( '/content/raw-c', exist_ok=True )

In [344]:
URL = "https://raw.githubusercontent.com/seantrott/raw-c/main/data/processed/raw-c.csv"
download = requests.get(URL).content
df = pd.read_csv(io.StringIO(download.decode('utf-8')),sep = ',')
#directory = URL.split('/')[4]

In [345]:
def getname(dataset):
  return dataset.split('.')[0]

In [442]:
url1 = "https://raw.githubusercontent.com/seantrott/raw-c/main/data/processed/normed_critical.csv"
dload = requests.get(URL).content
norm = pd.read_csv(io.StringIO(dload.decode('utf-8')),sep = ',')

In [349]:
df_ = pd.merge(df, norm, how = 'inner', on=['word','same', 'Class', 'ambiguity_type', 'version'])

In [351]:
df_dash = df_[['word', 'sentence1', 'sentence2', "Class", 'version']]
dataf = df[['word', 'sentence1', 'sentence2', "Class", 'version']]

In [352]:
def ver1(indd):
  return "_".join(indd.split("_", 2)[:2])

In [353]:
def ver2(indd):
  return "_".join(indd.split("_", 2)[2:])

In [354]:
df_dash['version1'] = df_dash['version'].apply(ver1)
dataf['version1'] = dataf['version'].apply(ver1)

In [355]:
df_dash['version2'] = df_dash['version'].apply(ver2)
dataf['version2'] = dataf['version'].apply(ver2)

In [356]:
df1 = df_dash[['word', 'sentence1', 'Class', 'version1']]
df1.columns =  ['word', 'sent', 'class', 'version']
df1["index"] = df1.groupby("word").cumcount()*2

In [357]:
df_1 = dataf[['word', 'sentence1', 'Class', 'version1']]
df_1.columns =  ['word', 'sent', 'class', 'version']
df_1["index"] = df_1.groupby("word").cumcount()*2

In [358]:
df2 =  df_dash[['word', 'sentence2', 'Class', 'version2']]
df2.columns =  ['word', 'sent', 'class', 'version']
df2["index"] = df2.groupby("word").cumcount()*2+1

In [359]:
df_2 =  dataf[['word', 'sentence2', 'Class', 'version2']]
df_2.columns =  ['word', 'sent', 'class', 'version']
df_2["index"] = df_2.groupby("word").cumcount()*2+1

In [360]:
df_final = pd.concat([df1, df2])
df_final = df_final.sort_values(by = ['word', 'index'], ascending = [True, True])

In [361]:
datafl = pd.concat([df_1, df_2])
datafl = datafl.sort_values(by = ['word', 'index'], ascending = [True, True])

In [362]:
df_final["identifier"] = df_final["word"]+"-"+df_final["version"].astype(str)

In [363]:
df_odd = df_final[1::2]
df_odd.columns = ["word2", "sent2", 'class2', 'version2', "index2", "identifier2"]
df_even = df_final[::2]
df_even.columns = ["word1", "sent1", "class1", 'version1', "index1", "identifier1"]
df_final = pd.concat([df_even, df_odd], axis=1)

In [364]:
datafl["identifier"] = datafl["word"]+"-"+datafl["version"].astype(str)

In [368]:
dataf_odd = datafl[1::2]
dataf_odd.columns = ["word2", "sent2", 'class2', 'version2', "index2", "identifier2"]
dataf_even = datafl[::2]
dataf_even.columns = ["word1", "sent1", "class1", 'version1', "index1", "identifier1"]
data_fl = pd.concat([dataf_even, dataf_odd], axis=1)

In [371]:
df_final = df_final[["word2", "sent2", "identifier2", 'class2',"sent1", "identifier1", 'class1', 'index1', 'index2', 'version2', 'version1']]
data_fl = data_fl[["word2", "sent2", "identifier2", 'class2',"sent1", "identifier1", 'class1', 'index2', 'index1', 'version2', 'version1']]
df_final_next_stage = data_fl.rename(columns={"word2":"lemma"})

In [372]:
data_fl = df_final_next_stage

In [374]:
df_final = df_final.rename(columns={"word2":"lemma"})

In [375]:
df_final["judgment"] = df_["relatedness"]

In [376]:
df_final = df_final[['lemma', 'identifier2', 'identifier1', 'judgment']]

In [377]:
df_final["annotator"] = df_["subject"]
df_final["comment"] = " "
df_final["description"] = df_['ambiguity_type']
#df_final['split'] = df_final['split2']

In [378]:
df_final = df_final[["identifier1", "identifier2", "annotator", "judgment", "comment", "lemma", "description"]]

In [380]:
for i in list(df_final["lemma"].value_counts().index):
  df_temp = df_final[df_final["lemma"]==i]
  numpy_df = df_temp.to_numpy()
  header = list(df_temp.columns)
  numpy_df = np.vstack([header, numpy_df])
  if not os.path.exists('/content/raw-c'+"/"+i):
      os.mkdir('/content/raw-c'+"/"+i)
  np.savetxt('/content/raw-c'+"/"+i+"/judgments.csv", numpy_df,fmt='%s', delimiter='\t')


In [400]:
df1 = df_final_next_stage[['lemma', 'sent1', 'class1', 'identifier1', 'index1']]
df1.columns =  ['word', 'sent', 'pos', 'identifier', 'index']
df2 = df_final_next_stage[['lemma', 'sent2', 'class2', 'identifier2', 'index2']]
df2.columns =  ['word', 'sent', 'pos', 'identifier', 'index']


In [401]:
df_final = pd.concat([df1, df2])

In [402]:
df_final = df_final.reset_index(drop = True)

In [403]:
from fuzzywuzzy import process
tag = ''
def get_indices_of_tags(sentence, word):
   i = sentence.split(" ")
   tag = process.extractOne(word, i)
   tag = tag[0]
   #print(tag)
   tag = re.sub('\W+', '', tag)
   return(str(sentence.find(tag))+":"+str(sentence.find(tag)+len(tag)))


In [404]:
def get_target(sentence, word):
   tok = sentence.split(" ")
   tag = process.extractOne(word, tok)
   tag = tag[0]
   #print(tag)
   for i in tok:
     if tag == i:
       ind = tok.index(i)
   return ind

In [405]:
df_final["indexes_target_token"] = df_final.apply(lambda x: get_indices_of_tags(x.sent, x.word), axis=1)

In [406]:
df_final['indexes_target_token_tokenized'] = df_final.apply(lambda x: get_target(x.sent, x.word), axis=1)

In [407]:
def get_indices_of_sent(sentence):
    return "0:"+str(len(sentence))

In [408]:
df_final["indexes_target_sentence"] = df_final["sent"].apply(get_indices_of_sent)

In [409]:
df_final = df_final.rename(columns = {"sent":"context", "word":"lemma"})

In [410]:
df_final['description'] = " "
df_final['date'] = " "
df_final['grouping'] = 1

In [411]:
df_final = df_final[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'indexes_target_token_tokenized', 'index']]

In [412]:
df_final['context_tokenized'] = df_final['context']

In [413]:
def get_len_tok(sentence):
  return "0:"+str(len(sentence.split(" ")))

In [414]:
df_final['indexes_target_sentence_tokenized'] = df_final["context_tokenized"].apply(get_len_tok)

In [415]:
lem = ''
def lemmatizr(sent):
  doc = nlp(sent)
  for i in doc:
    lem += i.lemma_
    return lem

In [416]:
df_final['context_lemmatized'] = df_final["context_tokenized"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [417]:
df_final['context_pos'] = df_final["context_tokenized"].apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))

In [418]:
final_df = df_final[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos', 'index']]

In [427]:
final_df = final_df.sort_values(by = ['lemma','index'], ascending =[True, True]).drop("index", axis=1)

In [428]:
final_df = final_df.reset_index(drop = True)

In [439]:
final_df = final_df.drop_duplicates().reset_index(drop = True)

In [440]:
for i in list(final_df["lemma"].value_counts().index):
  df_temp = final_df[final_df["lemma"]==i]
  numpy_df = df_temp.to_numpy()
  header = list(df_temp.columns)
  numpy_df = np.vstack([header, numpy_df])
  if not os.path.exists('/content/raw-c'+"/"+i):
      os.mkdir('/content/raw-c'+"/"+i)
  np.savetxt('/content/raw-c'+"/"+i+"/uses.csv", numpy_df,fmt='%s', delimiter='\t')


In [ ]:
#for i in list(final_df["lemma"].value_counts().index):
   # df_temp = final_df[final_df["lemma"]==i]
    #numpy_df = df_temp.to_numpy()
    #header = list(df_temp.columns)
    #numpy_df = np.vstack([header, numpy_df])
    #if not os.path.exists(i):
        #os.mkdir(i)
    #np.savetxt(i+"/uses.csv", numpy_df,fmt='%s', delimiter='\t')